<a href="https://colab.research.google.com/github/pcashman21/feral-cat-census/blob/main/arc/notebooks/generate_transformed_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses Tensorflow data augmentation to generate N transforms of each image in a directory.  The notebook creates a DataFrame with the following columns:

1.   **filename**: Name of file without path information, in the form imgXXX.jpg
2.   **label**: Label denoting which group this image belongs to.  Each cat and its transforms are a separate group.

For more information see https://www.tensorflow.org/tutorials/images/data_augmentation.

In [5]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
def transform_one_image(row, path_to_read, path_to_write, transformer, number_of_transforms):
    """
    This function takes in an image and creates a number of transforms of the image.

    row: row of dataframe with image name and usability
    path_to_read: path to folder of images
    path_to_write: path to folder to save transformed images
    transformer: image transformer object
    number_of_transforms: number of transforms to create
    Returns:
        dataframe with transformed images
    """

    image_file = row['filename']
    image_label = row['label']
    image_file_path = os.path.join(path_to_read,image_file)
    image = plt.imread(image_file_path)
    image_file_name = image_file.split('.')[0]
    image_file_extension = image_file.split('.')[1]
    image_file_name_transformed_root = image_file_name + '_transformed_'
    df_transform = pd.DataFrame({'filename':[], 'label': []})

    for i in range(number_of_transforms):
      # Generate name of transformed file
      image_file_name_transformed = image_file_name_transformed_root + str(i) + '.' + image_file_extension
      image_file_path_transformed = os.path.join(path_to_write, image_file_name_transformed)
      # Transform the image
      image_transformed = transformer(image)
      # Convert tensor to numpy array and save
      plt.imsave(image_file_path_transformed,image_transformed.numpy())
      gen_df = pd.DataFrame({'filename':[image_file_name_transformed], 'label': [image_label]})
      df_transform = pd.concat([df_transform, gen_df], axis=0, ignore_index=True)
    return df_transform

In [8]:
def get_image_files(path_to_read):
    """
    This function reads all image files in a folder and returns a list of file names.

    path_to_read: path to folder of images
    """

    image_files = []
    with os.scandir(path_to_read) as files:
      for file in files:
        last_component = file.name.split('.')[-1]
        if last_component == 'jpg' or last_component == 'jpeg':
          image_files.append(file.name)
    return image_files

In [9]:
def extract_label(image_file):
    """
    This function extracts the label from the image file name.
    The filenames are of the form 'imgXXX.jpg' where XXX is the label.

    image_file: name of image file
    """

    label = image_file.split('.')[0][3:]
    return label

In [20]:
def transform_images_folder(path_to_read, path_to_write, transformer, number_of_transforms):
    """
    This function reads all image files in a folder, creates a number of
    transforms of each image, and saves the transforms to a folder.

    path_to_read: path to folder of images
    path_to_write: path to folder to save transformed images
    transformer: image transformer object
    number_of_transforms: number of transforms to create

    Returns:
        updated dataframe with transformed images
    """

    image_files = get_image_files(path_to_read)
    df = pd.DataFrame({'filename':image_files, 'label': [extract_label(filename) for filename in image_files]})

    # Since transform_one_image returns a dataframe with multiple rows, we can't
    # use a lambda to iterate, as 'apply' will try to stuff the result in the row
    # which is the lambda argument.
    for i in range(len(df)):
      df_transformed = transform_one_image(df.iloc[i], path_to_read, path_to_write, transformer, number_of_transforms)
      df = pd.concat([df, df_transformed], axis=0, ignore_index=True)
    return df

In [11]:
IMG_SIZE = 244
NUMBER_OF_TRANSFORMS = 50

In [22]:
PATH_TO_READ = '/content/cat-face'
PATH_TO_WRITE = '/content/cat-face-transformed'
!mkdir $PATH_TO_WRITE


In [15]:
import zipfile
zip_archive = zipfile.ZipFile(PATH_TO_READ + '.zip', 'r')
zip_archive.extractall('/content')
zip_archive.close()


In [16]:
# Resize the input image, rescale the pixel values, and perform a series of transforms
transformer_nn = tf.keras.Sequential([
        layers.Resizing(IMG_SIZE, IMG_SIZE),
        layers.Rescaling(1./255),
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomTranslation(0.3, 0.3),
        layers.RandomZoom(0.2)
    ])

In [23]:
# Run the transform on all images
df = transform_images_folder(PATH_TO_READ, PATH_TO_WRITE, transformer_nn, NUMBER_OF_TRANSFORMS)
df.head()

,filename,label
0,img91.jpg,91
1,img60.jpg,60
2,img51.jpg,51
3,img11.jpg,11
4,img5.jpg,5


In [ ]:
!zip -r /content/cat-face-transformed.zip /content/cat-face-transformed/*.jpg



In [38]:
df.to_csv(PATH_TO_WRITE + '.csv', index=False)

